In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [ ]:
#pickle_file = 'cucntt_cuclaim_null_randomfix.pickle'
#pickle_file = 'rand01 outdel0 dummy0 tryno00 delcol-CUST.MAX_PRM.pickle'
#pickle_file = 'rand01 outdel0 dummy0 tryno00 delcol-CUST.MINCRDT.pickle'
#pickle_file = 'rand01 outdel0 dummy0 tryno00 delcol-CUST.WEDD_YN.pickle'
#pickle_file = 'clcucntt_rand01.pickle'
pickle_file = 'clcntt_randfix01.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  test_labels = save['test_label']
  test_dataset = save['test_data']
  train_labels = save['train_label']
  train_dataset = save['train_data']
  column_names = save['col_names']
  train_distri = save['train_distri']
  

del save  # hint to help gc free up memory
print('Test set : ', test_labels.shape, test_dataset.shape)
print('Training set : ', train_labels.shape, train_dataset.shape)


one hot encoding

In [ ]:
num_labels = 2

def reformat(labels):
  one_hot_encode = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return one_hot_encode 

test_labels=reformat(test_labels)
train_labels=reformat(train_labels)

print('Test set : ', test_labels.shape, test_dataset.shape)
print('Training set : ', train_labels.shape, train_dataset.shape)
print(train_dataset[:10])
print(train_labels[:100])

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def f1Score(predictions, labels):
    print ('predictions_raw=',predictions)
    predictions = np.array(predictions==predictions.max(axis=1)[:,None],dtype='int')
    predict = np.sum(predictions,0)
    real = np.sum(labels,0)
    correct_predict = np.sum(labels*(np.argmax(predictions, 1) == np.argmax(labels, 1))[:,None],0)
    precision = correct_predict[1]/predict[1]
    recall = correct_predict[1]/real[1]
    f1_score=2*precision*recall/(precision+recall)
    print ('predictions =',predictions)
    print ('predict =',predict)
    print ('real =',real)
    print ('correct_predict =',correct_predict)
    #여기부터 결과. 위는 디버깅용
    #print("\nsin predict",predict[1])
    #print("sin correct predict : ",correct_predict[1])
    #print("sin real : ",real[1])
    #print("\nnot_sin pricision : ",correct_predict[0]/predict[0])
    #print("not_sin   recall  : ",correct_predict[0]/real[0])
    #print("not_sin F1 score  : ",2*correct_predict[0]/predict[0]*correct_predict[0]/real[0]/(correct_predict[0]/predict[0]+correct_predict[0]/real[0]))
    print("sin pricision : ",precision)
    print("sin   recall  : ",recall)
    print("sin F1 score  : ",f1_score)

In [ ]:
#setting py
batch_size = 100
w_init_deviation = 0.5
learning_rate=0.4
layer2_nodes = 10
keep_ratio=1


step_size= train_dataset.shape[0]//batch_size
#print('step_size = ',step_size)

graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, train_dataset.shape[1]))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_test_dataset = tf.constant(test_dataset)
    
  L1_weights = tf.Variable(
    tf.truncated_normal([int(train_dataset.shape[1]), layer2_nodes], mean=0.0, stddev=w_init_deviation))
  L1_biases =  tf.Variable(tf.zeros([layer2_nodes]))
  L1_logits= tf.nn.dropout(tf.matmul(tf_train_dataset, L1_weights) + L1_biases,keep_ratio)
    
  L2_weights = tf.Variable(
    tf.truncated_normal([layer2_nodes, num_labels], mean=0.0, stddev=w_init_deviation))
  L2_biases = tf.Variable(tf.zeros([num_labels]))
  L2_logits=tf.matmul(tf.nn.relu(L1_logits), L2_weights) + L2_biases
    
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(L2_logits, tf_train_labels))
    
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
  train_prediction = tf.nn.softmax(L2_logits)
  test_prediction =tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(test_dataset, L1_weights) + L1_biases),L2_weights)+L2_biases)
    
  saver = tf.train.Saver({'w1' : L1_weights,
                          'b1' : L1_biases,
                          'w2' : L2_weights,
                          'b2' : L2_biases,
                         })

In [ ]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(step_size):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    global_step=step
    _,l, predictions = session.run(
      [optimizer,loss, train_prediction], feed_dict=feed_dict)
    if (step % (int(step_size*0.2)) == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
  #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
  f1Score(test_prediction.eval(), test_labels)
  saver.save(session,'tensorflow_wbsave frompk '+pickle_file[:-7],write_meta_graph=False)
  w1 = session.run(L1_weights)
  b1 = session.run(L1_biases)
  w2 = session.run(L2_weights)
  b2 = session.run(L2_biases)

  #print('test_prediction = 'session.run(test_prediction))#test 결과체크


pickle_name='wbsave frompk '+pickle_file
f=open(pickle_name,'w')
save ={'w1' : w1,
       'b1' : b1,
       'w2' : w2,
       'b2' : b2,
       'col_names' : np.array(column_names),#numpy.array 안붙이면 shape 못쓰기도 하고 다음에 꺼내쓸때 오류날수있음
       'tr_dist' : np.array(train_distri),
      }
pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
f.close()

#check pickle
print("\npicklize finished")
with open(pickle_name,'r') as f:
    trainwb=pickle.load(f)
    print(pickle_name)
    for k,v in trainwb.items():
        print(k,v.shape)




In [ ]:
# -*- coding: utf-8 -*-
import openpyxl
import os

#엑셀이름
excel_name=pickle_name[:pickle_name.find(".pickle")]+'.xlsx'

def pickleread(pickle_name):
    with open(pickle_name,'rb') as f:
        data=pickle.load(f)
        return data

def sheetmake(data):
    global pickle_name,excel_name
    book = openpyxl.Workbook()
    for dictitle , dictdata in data.items():
        dictdata=np.matrix(dictdata) #1차원 배열 있으면 shape 차원 하나라 오류나서.
        sheet=book.create_sheet(title=dictitle)
        for n_col in range(0,dictdata.shape[1]):
            for n_row in range(0,dictdata.shape[0]):
                input_value=np.asscalar(dictdata[n_row,n_col])          #python native 로 바꿔주는 코드. 이것과 아래줄 둘중하나 필수. 
                #input_value=dictdata[n_row,n_col]                           #str 오류날때 asscalar 빼면 될때있음
                sheet.cell(row=n_row+1,column=n_col+1).value=input_value    #엑셀에선 행,열 첫번호가 1 
            #sheet.column_dimensions[openpyxl.cell.get_column_letter(n_col+1)].width = 2.76 #컬럼 넓이 조절. 필요 없으면 빼기
        print('making sheet : ',dictitle)
    sheet = book.get_sheet_by_name('Sheet') #select sheet named Sheet
    book.remove_sheet(sheet) #delete that sheet
    print('saving data to excel...')
    book.save(excel_name)
    print ('finished, file saved : ',excel_name)



if os.path.isfile(excel_name): #이미 파일이 있으면 삭제함 #엑셀파일이 열려있으면 삭제도 못하고 오류남. 
	os.remove(excel_name)
	print('target excel file exists. continue after deleting')
sheetmake(pickleread(pickle_name))
